# Requisitos: pip install tensorflow, matplotlib, numpy

In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, LeakyReLU, BatchNormalization, Reshape, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt

In [4]:
# Cargar el dataset MNIST
(X_train, _), (_,_) = tf.keras.datasets.mnist.load_data()
X_train = X_train / 255.0 # Normalizar imagenes entre 0 y 1
X_train = X_train.reshape(-1,28,28,1).astype('float32')

In [6]:
# Tama;o dle espacio latente (ruido de entrada para el generador)
latent_dim = 100

In [7]:
# Crear el generador
def buil_generador():
    model = Sequential([
        Dense(256, input_dim=latent_dim),
        LeakyReLU(alpha=0.2),
        BatchNormalization(),
        Dense(512),
        LeakyReLU(alpha=0.2),
        BatchNormalization(),
        Dense(1024),
        LeakyReLU(alpha=0.2),
        BatchNormalization(),
        Dense(28*28*1, activation='tanh'),
        Reshape((28,28,1))
    ])
    return model

In [9]:
# Crear el Discriminador(filtro)
def build_discriminador():
    model = Sequential([
        Flatten(input_shape=(28,28,1)),
        Dense(512),
        LeakyReLU(alpha=0.2),
        Dense(1, activation='sigmoid') # Clasificacion entre real (1) o Falso (0)
    ])
    return model

In [10]:
# Compilar el modelo
discriminador = build_discriminador()
discriminador.compile(
    optimizer=Adam(learning_rate=0.0002, beta_1=0.5),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

C:\Users\gamer\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
C:\Users\gamer\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


In [11]:
# Compilar el generador (dentrode la GAN)
generador = buil_generador()

C:\Users\gamer\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\gamer\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


In [13]:
# Compilar el discriminador durante el entrenamiento del generador
discriminador.trainable=False
# Crear la GAN combinadoel generador y el discriminador
gan_input = tf.keras.Input(shape=(latent_dim,))
gan_output = discriminador(generador(gan_input))
gan = tf.keras.Model(gan_input, gan_output)

In [14]:
# Compilar la GAN
gan.compile(
    optimizer=Adam(learning_rate=0.002, beta_1=0.5),
    loss='binary_crossentropy'
)